In [11]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import pandas as pd
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import CorrectLevel, County, DataTrustedIdentifier, TrustedIdentifier, DataSP, Image, create_data_trusted_identifier

## Database connection

In [13]:
engine, session = db.connect()
engine.echo = False

2023-03-27 10:07:17,397 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-27 10:07:17,397 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 10:07:17,398 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-27 10:07:17,398 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-27 10:07:17,399 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-27 10:07:17,399 INFO sqlalchemy.engine.Engine [raw sql] {}


In [14]:
db.create_table(engine, CorrectLevel)

In [15]:
old_genus = [['Sarcorhachis'], ['Pothomorphe'], ['Piperomia', 'Peperonia']]
new_genus = ['Manekia', 'Piper', 'Peperomia']

for g in zip(old_genus, new_genus):
    list_old_genus = g[0]
    new = g[1]
    for old in list_old_genus:
        session.query(DataTrustedIdentifier)\
            .filter(DataTrustedIdentifier.genus.__eq__(old))\
            .update(values={DataTrustedIdentifier.genus_trusted: new}, synchronize_session=False)

In [41]:
df = pd.read_csv('../csv/list_genus_species_correct.csv', sep=';', index_col=None, header=0).astype(str)
df = df.replace('nan', None)
df

,kingdom,phylum,order,classe,family,genus,specific_epithet,infraspecific_epithet,scientific_name_authorship,kingdom_trusted,phylum_trusted,order_trusted,classe_trusted,family_trusted,genus_trusted,specific_epithet_trusted,infraspecific_epithet_trusted,scientific_name_authorship_trusted
0,None,None,None,None,None,Ottonia,anisum,None,Spreng.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
1,None,None,None,None,None,Ottonia,anisum,f. glabrior,Miq.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
2,None,None,None,None,None,Ottonia,armondii,None,Trel.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
3,None,None,None,None,None,Ottonia,burchellii,None,Trel.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
4,None,None,None,None,None,Ottonia,carpinifolia,f. hirtipedicellata,Yunck.,None,None,None,None,None,Piper,anisum,None,(Spreng.) Angely
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,None,None,None,None,None,Ottonia,brevistipitata,None,(C. DC.) Trel.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
114,None,None,None,None,None,Ottonia,leptostachya,var. brevistipitata,(C. DC.) E.F. Guim.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
115,None,None,None,None,None,Piper,frutescens,None,C. DC.,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.
116,None,None,None,None,None,Ottonia frutescens (C. DC.) Trel.,None,None,None,None,None,None,None,None,Piper,ovatum,var. hirtellum,C. DC.


In [43]:
session.commit()
session.flush()

def get_old_columns(columns):
    genus = columns.genus
    specific_epithet = columns.specific_epithet
    infraspecific_epithet = columns.infraspecific_epithet
    if infraspecific_epithet:
        infraspecific_epithet = infraspecific_epithet.replace('f. ', '').replace('var. ', '')
    scientific_name_authorship = columns.scientific_name_authorship
    if scientific_name_authorship:
        scientific_name_authorship = scientific_name_authorship.replace(' ', '').replace('(', '').replace(')','').replace('.', '')
    return genus, specific_epithet, infraspecific_epithet, scientific_name_authorship


def get_new_columns(columns):
    genus = columns.genus_trusted
    specific_epithet = columns.specific_epithet_trusted
    infraspecific_epithet = columns.infraspecific_epithet_trusted
    if infraspecific_epithet:
        infraspecific_epithet = infraspecific_epithet.replace('f. ', '').replace('var. ', '')
    scientific_name_authorship = columns.scientific_name_authorship_trusted
    if scientific_name_authorship:
        scientific_name_authorship = scientific_name_authorship.replace(' ', '').replace('(', '').replace(')','').replace('.', '')
    return genus, specific_epithet, infraspecific_epithet, scientific_name_authorship


for d in df[['genus', 'specific_epithet', 'infraspecific_epithet', 'scientific_name_authorship', 'genus_trusted', 'specific_epithet_trusted', 'infraspecific_epithet_trusted','scientific_name_authorship_trusted']].iterrows():
    print('genus (old) %s - (new) %s' % (d[1].genus, d[1].genus_trusted))
    print('specific_epithet (old) %s - (new) %s' % (d[1].specific_epithet, d[1].specific_epithet_trusted))
    print('infraspecific_epithet (old) %s - (new) %s' % (d[1].infraspecific_epithet, d[1].infraspecific_epithet_trusted))
    print('scientific_name_authorship (old) %s - (new) %s' % (d[1].scientific_name_authorship, d[1].scientific_name_authorship_trusted))

    print(type(d[1].infraspecific_epithet_trusted))


    # sub = session.query(DataTrustedIdentifier)\
    #     .filter(sa.and_(DataTrustedIdentifier.genus.__eq__(d[1].genus),
    #                     DataTrustedIdentifier.specific_epithet.__eq__(d[1].specific_epithet),
    #                     sa.or_(DataTrustedIdentifier.infraspecific_epithet.__eq__(d[1].infraspecific_epithet), DataTrustedIdentifier.scientific_name_authorship.__eq__(d[1].scientific_name_authorship))))\
    #     .subquery('sub')

    session.query(DataTrustedIdentifier)\
        .filter(sa.and_(DataTrustedIdentifier.genus.__eq__(d[1].genus),
                        DataTrustedIdentifier.specific_epithet.__eq__(d[1].specific_epithet),
                        sa.or_(DataTrustedIdentifier.infraspecific_epithet.__eq__(d[1].infraspecific_epithet), DataTrustedIdentifier.scientific_name_authorship.__eq__(d[1].scientific_name_authorship))))\
        .update(values={DataTrustedIdentifier.genus_trusted: d[1].genus_trusted,
                        DataTrustedIdentifier.specific_epithet_trusted: d[1].specific_epithet_trusted,
                        DataTrustedIdentifier.infraspecific_epithet_trusted: d[1].infraspecific_epithet_trusted,
                        DataTrustedIdentifier.scientific_name_authorship_trusted: d[1].scientific_name_authorship_trusted}, synchronize_session=False)

    session.commit()
    break

genus (old) Ottonia - (new) Piper
specific_epithet (old) anisum - (new) anisum
infraspecific_epithet (old) None - (new) None
scientific_name_authorship (old) Spreng. - (new) (Spreng.) Angely
<class 'NoneType'>


In [ ]:
# genus_old, specific_epithet_old, infraspecific_epithet_old, scientific_name_authorship_old = get_old_columns(d[1])
    #
    # genus_new, specific_epithet_new, infraspecific_epithet_new, scientific_name_authorship_new = get_new_columns(d[1])
    #
    #
    # column_infraspecific_epithet = sa.func.replace(
    #     sa.func.replace(DataTrustedIdentifier.infraspecific_epithet, 'f. ', ''),
    #     'var. ', '')
    # column_scientific_name_authorship = sa.func.replace(
    #     sa.func.replace(
    #         sa.func.replace(
    #             DataTrustedIdentifier.scientific_name_authorship, '.', ''),
    #         '(', ''),
    #     ')', '')
    #
    # columns = [
    #     DataTrustedIdentifier.genus,
    #     DataTrustedIdentifier.specific_epithet,
    #     column_infraspecific_epithet.label('infraspecific_epithet'),
    #     column_scientific_name_authorship.label('scientific_name_authorship'),
    # ]
    #
    # sub = session.query(*columns) \
    #     .filter(sa.and_(DataTrustedIdentifier.genus.__eq__(genus_old),
    #                     DataTrustedIdentifier.specific_epithet.__eq__(specific_epithet_old))) \
    #     .subquery('sub')
    #
    # session.query(DataTrustedIdentifier) \
    #     .filter(sa.and_(sub.c.genus.__eq__(genus_old),
    #                     sub.c.specific_epithet.__eq__(specific_epithet_old),
    #                     sa.or_(sub.c.infraspecific_epithet.__eq__(infraspecific_epithet_old),
    #                            sub.c.scientific_name_authorship.__eq__(scientific_name_authorship_old))))\
    #     .update(values={DataTrustedIdentifier.genus_trusted: genus_new,
    #                     DataTrustedIdentifier.specific_epithet_trusted: specific_epithet_new,
    #                     DataTrustedIdentifier.infraspecific_epithet_trusted: infraspecific_epithet_new,
    #                     DataTrustedIdentifier.scientific_name_authorship_trusted: scientific_name_authorship_new},
    #             synchronize_session=False)
    # session.commit()

## Close connection

In [18]:
session.close()
engine.dispose()